In [84]:
import pandas as pd
import os
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import json



from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import re

from sklearn.linear_model import SGDClassifier

In [61]:
def regex_condi(string):
    if ".com" in string:
        return False
    if ".edu" in string:
        return False
    if "@" in string:
        return False
    if 'Host'in string:
        return False
    if ".gov" in string:
        return False

    return True

def clean_text(string):
    string = " ".join([i for i in string.split() if regex_condi(i)])
    string = re.sub(r"Re:","", string)
    string = re.sub(r"Reply-To:","", string)
    toreturn = string.strip()
    return toreturn

In [77]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

newsgroups_train_X,newsgroups_train_y = newsgroups_train.data, newsgroups_train.target
newsgroups_val_X, newsgroups_test_X, newsgroups_val_y, newsgroups_test_y = train_test_split(newsgroups_test.data, newsgroups_test.target, test_size=0.5, random_state=42)


# Clean train 
newsgroups_train_X = [clean_text(i) for i in newsgroups_train_X]
newsgroups_val_X = [clean_text(i) for i in newsgroups_val_X]
newsgroups_test_X =  [clean_text(i) for i in newsgroups_test_X]

In [129]:
def BoG_model(X,y, clf = 'Logistic', vocab = None):
    """
    This function take in training data as X, y. 
    Then the data will be processed with BagOfWord, then feed into the Logistic / SGD Classifier.

    Return:
        pipe: The fitted Logistic Classifier.
    """
    if vocab == None:
        if clf == 'Logistic':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer()),
               ('clasiffier', LogisticRegression())
               ])
        if clf == 'SVM':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer()),
               ('clasiffier', SGDClassifier())
               ])
    else:
        if clf == 'Logistic':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer(vocabulary =vocab)),
               ('clasiffier', LogisticRegression())
               ])
        if clf == 'SVM':
            pipe = Pipeline([
               ('BagOfWord', CountVectorizer(vocabulary =vocab)),
               ('clasiffier', SGDClassifier())
               ])
    pipe.fit(X, y)
    return pipe

def Tfidf_model(X,y, clf = 'Logistic', vocab = None):

    """
    This function take in training data as X, y. 
    Then the data will be processed with BagOfWord and Tf-Idf, then feed into the Logistic Classifier.

    Return:
        pipe: The fitted Logistic Classifier.
    """
    if vocab == None:
        if clf == 'Logistic':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer()),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', LogisticRegression())
                ])
        if clf == 'SVM':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer()),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', SGDClassifier())
                ])
    else:
        if clf == 'Logistic':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer(vocabulary =vocab)),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', LogisticRegression())
                ])
        if clf == 'SVM':
            pipe = Pipeline([
                ('BagOfWord', CountVectorizer(vocabulary =vocab)),
                ('TfIdf',TfidfTransformer()),
                ('clasiffier', SGDClassifier())
                ])
    pipe.fit(X, y)
    return pipe





In [118]:
Log_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y)
predicted = Log_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

C:\Users\Yang Li\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7774827403080191

In [119]:
Log_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y)
predicted = Log_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8266064790228359

In [121]:
SVM_Count_clf = BoG_model(newsgroups_train_X, newsgroups_train_y, clf = 'SVM')
predicted = SVM_Count_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.7525225703664365

In [123]:
SVM_TFIDF_clf = Tfidf_model(newsgroups_train_X, newsgroups_train_y, clf = 'SVM')
SVM_TFIDF_clf.fit(newsgroups_train_X, newsgroups_train_y)
predicted = SVM_TFIDF_clf.predict(newsgroups_val_X)
np.mean(predicted == newsgroups_val_y)

0.8454593733404142

In [92]:
os.makedirs('../data/temp',exist_ok=True)

In [97]:
news_df = pd.DataFrame.from_dict(newsgroups_train,'index').T
news_df.data

0        From: lerxst@wam.umd.edu (where's my thing)\nS...
1        From: guykuo@carson.u.washington.edu (Guy Kuo)...
2        From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3        From: jgreen@amber (Joe Green)\nSubject: Re: W...
4        From: jcm@head-cfa.harvard.edu (Jonathan McDow...
                               ...                        
11309    From: jim.zisfein@factory.com (Jim Zisfein) \n...
11310    From: ebodin@pearl.tufts.edu\nSubject: Screen ...
11311    From: westes@netcom.com (Will Estes)\nSubject:...
11312    From: steve@hcrlgw (Steven Collins)\nSubject: ...
11313    From: gunning@cco.caltech.edu (Kevin J. Gunnin...
Name: data, Length: 11314, dtype: object

In [98]:
with open('../data/temp/20news.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(news_df.data.apply(lambda x: clean_text(x)+ '.')))

In [127]:
df = pd.read_csv('../data/temp/AutoPhrase.txt',error_bad_lines=False, delimiter= '\t',names = ['p','phrase'])
select = df[df.p >= 0.5]
df

,p,phrase
0,0.988864,george bush
1,0.987358,red sox
2,0.986398,attorney general
3,0.985885,gulf war
4,0.985655,silicon graphics
...,...,...
12030,0.003707,not exactly
12031,0.003279,can't do
12032,0.003142,r +
12033,0.003109,probably not


In [126]:
count_vect = CountVectorizer(vocabulary = select.phrase,stop_words = 'english')
count_vect = count_vect.fit_transform(newsgroups_train_X)